# Locate and link notebook for vibration thin layer experiment

## A) Locate 
1) Import and read the .cine movie file from the experiment.

2) Create circular masks to focus only on the circular vibrating platform.

3) Apply the mask to the frames of the clip.

4) Locate all the features (particules) and create a DataFrame with x,y positions for each frame.

5) Save the DataFrame into a pickle file.



### Import the required libraries

In [ ]:
# -*- coding: utf-8 -*-
"""
Spyder Editor

This is a temporary script file.
"""
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import DataFrame, Series  
import pims
import trackpy as tp
#interactive graphics
%matplotlib widget 

from mpl_toolkits.axes_grid1 import make_axes_locatable
from IPython.display import Image

### Create circular masks to focus only on the circular vibrating platform

In [ ]:
def createCircularMask(h, w, center=None, radius=None):
    """
    Creates an OpenCV circular mask
        
    Parameters
    ----------
    h : int
    Height of the image for which the mask is going to be used
    w : int
    Width of the image for which the mask is going to be used
    center : tuple or list, optional
    Pair of coordinates for the mask's central point. If not specified uses: [w/2, h/2]
    radius : float, optional
    Value of the mask radius. If not specified uses max possible value
    
    Returns
    -------
    mask : array
    Mask for using with the image (array of True/False values)
    """
    
    if center is None: # use the middle of the image
        center = [int(w/2), int(h/2)]
    if radius is None: # use the smallest distance between the center and image walls
        radius = min(center[0], center[1], w-center[0], h-center[1])
    
    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)
    
    mask = dist_from_center <= radius
    
    return mask

@pims.pipeline
def maskImage(img, mask):
    """ 
    Masks an input image
      
    Parameters
    ----------
    img : array
    Input image
    mask : array
    True/False array with same shape as input image 
    
    Returns
    -------
    masked_img : array
    output masked image
        
    """
    masked_img = img.copy()
    masked_img[~mask] = 0
    
    return masked_img

### Import and read the .cine movie file from the experiment

Open the .cine file with PIMS

frames: cine object with all the frames of the movie
w, h define the width and height of the frames

In [ ]:
frames=pims.open('/home/juan/Desktop/21549_1_10.cine')
w,h=frames[0].shape


#plt.imshow(frames[0],cmap='gray')

### Apply the circular mask to the frames

It is required some try-and-check steps to apply rightly the mask to the circular platform.

PARAMETERS for the mask: 
* w, h
* center of the frame where the circular mask is applied
* radius of the mask

In [ ]:
mascara=createCircularMask(w,h,center=(416,399),radius=388);
imagen=maskImage(frames, mascara);
plt.figure()
plt.imshow(imagen[0],cmap='gray')
#f=tp.locate(imagen[0],5,minmass=115,separation=4)
#plt.figure()
#tp.annotate(f,imagen[0])
#plt.figure()
#plt.plot(f['mass'],'.')

### Locate the features in each frame

Features in each frame are the particles. Every particle have a unique (x,y) position for each frame (time instant).

The trackpy library is used to locate the particles. A for-loop is used to apply locate function for the whole frame banch.

A DataFrame is generated with columns=('x','y','frame').

PARAMETERS required: trackpy.locate(frame[i],size,minmass,separation)

* individual frame (frame[i])
* feature diamater size in pixels
* minimum mass of the features
* minimum separation in pixels

In [ ]:
df=pd.DataFrame()
for i in range(frames.shape[0]):
   if i==0:
       f=tp.locate(imagen[i],5,minmass=115,separation=4)
       f=f[['x','y','frame']]
       df=pd.concat([df,f], ignore_index=True)
       
       #f.to_csv('dataloc_120Hz_150us_75.csv',mode='a',index=False,header=True,columns=('x','y','frame'))
   else:
       leng=f.shape[0]
       f=tp.locate(imagen[i],5,minmass=115,separation=4)
       f=f[['x','y','frame']]
       df=pd.concat([df,f], ignore_index=True)
       #f.to_csv('dataloc_120Hz_150us_75.csv',mode='a',index=False,header=False,columns=('x','y','frame'))        
   if i%100==0:
       print(i)

### Save the DataFrame with all the particles located in all frames

A pickle file is used to save the DataFrame to save memory space

In [ ]:
df.to_pickle('/home/juan/OneDrive/Investigación 2022_Física/Laboratorio/data_viernes.pkl')

## B) Linking

This part of the notebook link all the particles along the frames and create trajectories for each particle.

1) Load DataFrame with the particle (x,y) location for each frame.

2) Calculate the number of frames of the movie.

3) Define the size of the sub DF to link the particles (this should save RAM and make the process quicker).

4) Apply link function from trackpy.

5) Save the Dataframe with trajectories for each particle along the frames.


### Load DataFrame from location
If these linking cells will be run without running locate cells, the dataframe of the location have to be loaded and create a dataframe variable

In [ ]:
df=pd.read_pickle('datalocate_2000fps_150us_75.pkl')

### Previous calculations

* Extract the number of frames
* Define the size of the sub-DF to link particles

In [ ]:
N=6
Nframe=np.max(df.frame.values)

k=round(Nframe/N)

### Apply link function from trackpy

tp.link (df,range, memory)

In [ ]:
linked=pd.DataFrame()
for i in range(k):
    sub=df[df.frame<(i+1)*N]
    sub=sub[sub.frame>=i*N]
    
  
#    globals()[f"linked{i}"]=tp.link(sub,4,memory=5)
    sublinked=tp.link(sub,4,memory=5)
    linked=pd.concat([linked,sublinked],ignore_index=True)

### Save the dataframe with trajectories

In [ ]:
linked.to_pickle('datalinked_2000fps_150us_75.pkl')
  
 #   linked.to_pickle(f'datalink_120HZ_150us_{i}.pkl')